In [1]:
from collections import Counter
import re

import numpy as np
import pandas as pd

In [2]:
DATA = '../../../../data'

# Part 1: Data Preprocessing 

As in any other machine learning task, the first thing you have to do is process your data set. 
- Many courses load in pre-processed data for you. 
- However, in the real world, when you build these NLP systems, you load the datasets and process them.
- So let's get some real world practice in pre-processing the data!

Your first task is to read in a file called **'shakespeare.txt'** which is found in your file directory. To look at this file you can go to `File ==> Open `. 

### Exercise 1
Implement the function `process_data` which 

1) Reads in a corpus (text file)

2) Changes everything to lowercase

3) Returns a list of words. 

#### Options and Hints
- If you would like more of a real-life practice, don't open the 'Hints' below (yet) and try searching the web to derive your answer.
- If you want a little help, click on the green "General Hints" section by clicking on it with your mouse.
- If you get stuck or are not getting the expected results, click on the green 'Detailed Hints' section to get hints for each step that you'll take to complete this function.

In [3]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just 
          have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file 
          you read) in lower case. 
    """
    with open(file_name, 'r') as f:
        text = f.read()
    text = re.sub('\W+', ' ', text)
    words = text.lower().split()
    return words

In [4]:
word_l = process_data(f'{DATA}/shakespeare.txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


### Exercise 2

Implement a `get_count` function that returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

For example, given the following sentence: **"I am happy because I am learning"**, your dictionary should return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>


**Instructions**: 
Implement a `get_count` which returns a dictionary where the key is a word and the value is the number of times the word appears in the list.  

In [5]:
def get_count(word_list):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word 
          and value is its frequency.
    '''
    word_count_dict = Counter(word_list)
    return word_count_dict

In [6]:
#DO NOT MODIFY THIS CELL
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


### Exercise 3
Given the dictionary of word counts, compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} \tag{Eqn-2}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7} \tag{Eqn-3}.$$

**Instructions:** Implement `get_probs` function which gives you the probability 
that a word occurs in a sample. This returns a dictionary where the keys are words, and the value for each word is its probability in the corpus of words.

In [7]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word 
          and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are 
          the probability that a word will occur. 
    '''
    M = sum(word_count_dict.values())
    probs = {k: v / M for k, v in word_count_dict.items()}
    return probs

In [8]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


# Part 2: String Manipulations

Now, that you have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, you will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


### Exercise 4

**Instructions for delete_letter():** Implement a `delete_letter()` function that, given a word, returns a list of strings with one character deleted. 

For example, given the word **nice**, it would return the set: {'ice', 'nce', 'nic', 'nie'}. 

**Step 1:** Create a list of 'splits'. This is all the ways you can split a word into Left and Right: For example,   
'nice is split into : `[('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')]`
This is common to all four functions (delete, replace, switch, insert).

**Step 2:** This is specific to `delete_letter`. Here, we are generating all words that result from deleting one character.  
This can be done in a single line with a list comprehension. You can make use of this type of syntax:  
`[f(a,b) for a, b in splits if condition]`  

For our 'nice' example you get: 
['ice', 'nce', 'nie', 'nic']

In [9]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible
          words in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 
          character from word
    '''
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    deletes = [''.join([a, b[1:]]) for a, b in splits if b]
    if verbose: print(f'input word {word}, \nsplit_l = {splits}, '
                      f'\ndelete_l = {deletes}')
    return deletes

In [10]:
word = 'cans'
letters = list(word)
splits = [(word[:i], word[i:]) for i in range(len(word))]
dels = [''.join([a, b[1:]]) for a, b in splits if b]
dels

['ans', 'cns', 'cas', 'can']

In [11]:
delete_word_l = delete_letter(word="cans", verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [12]:
print(f"Number of outputs of delete_letter('at') is "
      f"{len(delete_letter('at'))}")

Number of outputs of delete_letter('at') is 2


### Exercise 5

**Instructions for switch_letter()**: Now implement a function that switches two letters in a word. It takes in a word and returns a list of all the possible switches of two letters **that are adjacent to each other**. 
- For example, given the word 'eta', it returns {'eat', 'tea'}, but does not return 'ate'.

**Step 1:** is the same as in delete_letter()  
**Step 2:** A list comprehension or for loop which forms strings by swapping adjacent letters. This is of the form:  
`[f(L,R) for L, R in splits if condition]`  where 'condition' will test the length of R in a given iteration. See below.

In [13]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent 
        charater switched
    ''' 
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    swaps = [a + b[1] + b[0] + b[2:] for (a, b) in splits
             if len(b) > 1]
    if verbose: 
        print(f'Input word = {word} \nsplit_l = {splits} '
              f'\nswitch_l = {swaps}') 
    return swaps

In [14]:
swap_words = switch_letter(word='eta', verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [15]:
print(f"Number of outputs of switch_letter('at') is "
      f"{len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


### Exercise 6
**Instructions for replace_letter()**: Now implement a function that takes in a word and returns a list of strings with one **replaced letter** from the original word. 

**Step 1:** is the same as in `delete_letter()`

**Step 2:** A list comprehension or for loop which form strings by replacing letters.  This can be of the form:  
`[f(a,b,c) for a, b in splits if condition for c in string]`   Note the use of the second for loop.  
It is expected in this routine that one or more of the replacements will include the original word. For example, replacing the first letter of 'ear' with 'e' will return 'ear'.

**Step 3:** Remove the original input letter from the output.

In [51]:
a = ['c', 'a', 'n']
a.remove('a')
a

['c', 'n']

In [54]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one 
          letter from the original word. 
    ''' 
    
    letters = list('abcdefghijklmnopqrstuvwxyz')
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    replacements = set([''.join([a, x, b[1:]]) for (a, b) in splits
                        for x in letters])
    replacements.discard(word)
    replacements = sorted(list(replacements))
    if verbose: 
        print(f'Input word = {word} \nsplit_l = {splits} '
              f'\nreplace_l {replacements}')   
    return replacements

In [55]:
replacements = replace_letter(word='can', verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


### Exercise 7

**Instructions for insert_letter()**: Now implement a function that takes in a word and returns a list with a letter inserted at every offset.

**Step 1:** is the same as in `delete_letter()`

**Step 2:** This can be a list comprehension of the form:  
`[f(a,b,c) for a, b in splits if condition for c in string]`   

In [18]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = list('abcdefghijklmnopqrstuvwxyz')
    splits = [(word[:i], word[i:]) for i in range(len(word))]
    inserts = [''.join([a, x, b]) for (a, b) in splits for x in letters]
    inserts +=  [''.join([word, x]) for x in letters]
    if verbose: 
        print(f'Input word {word} \nsplit_l = {splits} '
              f'\ninsert_l = {inserts}')
    return inserts

In [19]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is "
      f"{len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


In [20]:
26  * 4

104

# Part 3: Combining the edits

Now that you have implemented the string manipulations, you will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

## 3.1 Edit one letter

### Exercise 8

**Instructions**: Implement the `edit_one_letter` function to get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation. You should use the previous functions you have already implemented to complete this function. The 'switch' function  is a less common edit function, so its use will be selected by an "allow_switches" input argument.

Note that those functions return *lists* while this function should return a *python set*. Utilizing a set eliminates any duplicate entries.

In [59]:
def edit_one_letter(word, allow_switches=True):
    """
    Input:
        word: the string/word for which we will generate all possible 
          words that are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return
        a set and not a list.
    """
    edits = set(delete_letter(word))
    edits = edits.union(set(replace_letter(word)))
    edits = edits.union(set(insert_letter(word)))
    if allow_switches:
        edits = edits.union(set(switch_letter(word)))
    return edits

In [60]:
tmp_word = 'at'
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f'input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n')
print(f'The type of the returned object should be a set '
      f'{type(tmp_edit_one_set)}')
print(f'Number of outputs from edit_one_letter("at") is '
      f'{len(edit_one_letter("at"))}')

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter("at") is 129


## Part 3.2 Edit two letters

### Exercise 9

Now you can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again. 

**Instructions**: Implement the `edit_two_letters` function that returns a set of words that are two edits away. Note that creating additional edits based on the `edit_one_letter` function may 'restore' some one_edits to zero or one edits. That is allowed here. This accounted for in get_corrections.

In [63]:
def edit_two_letters(word, allow_switches=True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''    
    edits_two = set()
    edits_one = edit_one_letter(word, allow_switches)
    for edit in edits_one:
        edits_two = edit_two_set.union(edit_one_letter(edit))
    return edit_two

In [65]:
tmp_edit_two_set = edit_two_letters('a')
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(
    f'Number of strings with edit distance of two: {len(tmp_edit_two_l)}')
print(f'First 10 strings {tmp_edit_two_l[:10]}')
print(f'Last 10 strings {tmp_edit_two_l[-10:]}')
print(f'The data type of the returned object should be a set '
      '{type(tmp_edit_two_set)}')
print(f"Number of strings that are 2 edit distances from 'at' is "
      f"{len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set {type(tmp_edit_two_set)}
Number of strings that are 2 edit distances from 'at' is 7154


## Part 3-3: suggest spelling suggestions

Now you will use your `edit_two_letters` function to get a set of all the possible 2 edits on your word. You will then use those strings to get the most probable word you meant to type aka your typing suggestion.

### Exercise 10
**Instructions**: Implement `get_corrections`, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 

**Step 1:** Generate suggestions for a supplied word: You'll use the edit functions you have developed. The 'suggestion algorithm' should follow this logic: 
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use those. 
* Otherwise, suggest the input word.*  
* The idea is that words generated from fewer edits are more likely than words with more edits.


Note: 
- Edits of one or two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

The logical `or` could be used to implement the suggestion algorithm very compactly. Alternately, if/then constructs could be used.
 
**Step 2**: Create a 'best_words' dictionary where the 'key' is a suggestion and the 'value' is the probability of that word in your vocabulary. If the word is not in the vocabulary, assign it a probability of 0.

**Step 3**: Select the n best suggestions. There may be fewer than n.

In [142]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    # NOTE: this is not very efficient, but has to be done stupidly to
    # satisfy the requirements of the course.  Once `edit_one_letter` has
    # been run, it reduplicates efforts to then run `edit_two_letters`    
    suggestions = []
    n_best = []
    def sort_suggestions(suggestions, n):
        mx = len(suggestions)
        suggestions = sorted(
            suggestions, key=lambda x: x[1], reverse=True)
        n = min(n, mx)
        return suggestions[:n]
    
    if word in probs:
        suggestions.append((word, probs[word]))
        n_best = suggestions
    if not n_best:
        for edit in edit_one_letter(word):
            if edit in probs:
                suggestions.append((edit, probs[edit]))
        n_best = sort_suggestions(suggestions, n)
    if not n_best:
        for edit in edit_two_letters(word):
            if edit in probs:
                suggestions.append((edit, probs[edit]))
        n_best = sort_suggestions(suggestions, n)
    n_best = n_best or [(word, 0)]
    suggestions = set([tup[0] for tup in suggestions])
    if verbose: 
        print('entered word =', word, '\nsuggestions = ', suggestions)
    return n_best

In [143]:
# Test your implementation - feel free to try other words in my word
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f'word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}')

print(f'data type of corrections {type(tmp_corrections)}')

entered word = dys 
suggestions =  {'days', 'dye'}
word 0: days, probability 0.000410
word 1: dye, probability 0.000019
data type of corrections <class 'list'>
